# Combine & Summarization

### 필요한 라이브러리 임포트

In [28]:
# !pip install lexrankr 
# !pip install kss 
# !pip install gensim

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import rhinoMorph
from sklearn.cluster import KMeans

### 3개의 csv loading

In [95]:
filename1 = 'articles_final_peter'
filename2 = 'extData_LDA_ver2'
filename3 = 'Willver3_rhino10'

def combinecsv (filename1, filename2, filename3):
    
    peter_df = pd.read_csv('./data/'+str(filename1)+'.csv')
    jm_df = pd.read_csv('./data/'+str(filename2)+'.csv')
    will_df = pd.read_csv('./data/'+str(filename3)+'.csv')
    
    df = pd.concat([peter_df,jm_df,will_df])
    df.sort_values(by='date',inplace=True)
    df.reset_index(inplace=True)
    df.drop('index',inplace=True,axis=1)
    return df

In [96]:
combined_df = combinecsv (filename1, filename2, filename3)
combined_df

,media,date,title,article_original,url
0,조선일보,2020-01-22,방탄소년단 4월 월드투어 돌입 서울부터 스페인까지,방탄소년단BTS이 오는 4월 서울을 시작으로 세계 투어에 돌입한다. 방탄소년단은 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-27,방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전,그룹 방탄소년단BTS이 26일현지 시각 한국 가수 최초로 미국 그래미 시싱식에서 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-29,7명의 몸짓을 예술로 현대미술이 BTS라는 물길을 만났다,커넥트 BTS서울 BTS 철학 담은 현대미술 작품 세계 5개도시서 펼치는 프로젝...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-02-22,방탄소년단 새 앨범 91개국 아이튠즈 1위 역대 최고기록,그룹 방탄소년단BTS 새 앨범이 발매하자마자 세계 91개 지역 아이튠즈 차트 1위...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,경향신문,2020-02-28,BTS 월드투어 4월 서울 콘서트 취소 코로나19 확산 비상,일부 팬들 환불받아 성금 기탁 트와이스 등 공연 취소 잇따라 코로나19의 확산으...,https://news.naver.com/main/read.nhn?mode=LSD&...
5,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔 7MAP OF THE SOUL 7...,https://news.naver.com/main/read.nhn?mode=LSD&...
6,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단BTS이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 차...,https://news.naver.com/main/read.nhn?mode=LSD&...
7,경향신문,2020-03-27,방탄소년단 코로나19 확산에 북미 투어 연기,그룹 방탄소년단BTS이 코로나19 확산으로 북미 투어 일정을 연기한다. 소속사 빅...,https://news.naver.com/main/read.nhn?mode=LSD&...
8,경향신문,2020-04-09,방탄소년단 음반 누적판매 2032만장한국 가요 최다 기록,그룹 방탄소년단사진이 앨범 누적 판매량 2000만장을 넘기면서 한국 가요 역사상 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
9,조선일보,2020-05-18,이태원 아이돌 BTS 정국 아스트로 차은우 NCT 재현 세븐틴 민규였다,최근 소문으로만 떠돌던 이태원발 코로나 집단 감염 기간 이태원 방문 아이돌이 그룹...,https://news.naver.com/main/read.nhn?mode=LSD&...


### 토크나이징 with rhinoMorph (feat. Stopwords)

In [97]:
StopWordsDir = "./data/stopwords.txt"


def tokenize(df, StopWordsDir):
    with open (StopWordsDir, "r", encoding = 'utf-8') as myfile:
        data=myfile.read()
    stop_words=data.split('\n')
    
    rhino = rhinoMorph.startRhino()
    df['tokenized_sents'] = df.article_original.apply(lambda x: rhinoMorph.onlyMorph_list(rhino, x, pos=['NNG','NNP','VA'], combineN=True)) # eomi=True, xrVv=True))
    df.tokenized_sents = df.tokenized_sents.apply(lambda x : [w for w in x if x not in stop_words if len(w) >1])  # 불용어 제거

    return df

In [98]:
tokenized_df = tokenize(combined_df, StopWordsDir)
tokenized_df

filepath:  C:\Users\user\anaconda3\Lib\site-packages
classpath:  C:\Users\user\anaconda3\Lib\site-packages\rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!


,media,date,title,article_original,url,tokenized_sents
0,조선일보,2020-01-22,방탄소년단 4월 월드투어 돌입 서울부터 스페인까지,방탄소년단BTS이 오는 4월 서울을 시작으로 세계 투어에 돌입한다. 방탄소년단은 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[방탄, 서울, 시작, 세계, 투어, 방탄소년단, 오전, 빅히트, 공식, SNS, ..."
1,조선일보,2020-01-27,방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전,그룹 방탄소년단BTS이 26일현지 시각 한국 가수 최초로 미국 그래미 시싱식에서 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[그룹, 방탄, 현지, 시각, 한국, 가수, 최초, 미국, 방탄소년단, 이날, 미국..."
2,조선일보,2020-01-29,7명의 몸짓을 예술로 현대미술이 BTS라는 물길을 만났다,커넥트 BTS서울 BTS 철학 담은 현대미술 작품 세계 5개도시서 펼치는 프로젝...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[서울, 철학, 현대미술, 작품, 세계, 도시, 프로젝트, 서울, 개막, 강이, 참..."
3,조선일보,2020-02-22,방탄소년단 새 앨범 91개국 아이튠즈 1위 역대 최고기록,그룹 방탄소년단BTS 새 앨범이 발매하자마자 세계 91개 지역 아이튠즈 차트 1위...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[그룹, 앨범, 세계, 지역, 아이, 차트, 방탄소년단, 앨범, 역대, 최고, 기록..."
4,경향신문,2020-02-28,BTS 월드투어 4월 서울 콘서트 취소 코로나19 확산 비상,일부 팬들 환불받아 성금 기탁 트와이스 등 공연 취소 잇따라 코로나19의 확산으...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[일부, 환불, 성금, 기탁, 트와이스, 공연, 취소, 코로나, 확산, 그룹, 방탄..."
5,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔 7MAP OF THE SOUL 7...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[그룹, 방탄, 앨범, 오브, 영국, 오피셜, 차트, 빌보드, 차트, 한국, 가수,..."
6,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단BTS이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 차...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[그룹, 방탄, 미국, 빌보드, 메인, 앨범, 차트, 빌보드, 연속, 빌보드, 예고..."
7,경향신문,2020-03-27,방탄소년단 코로나19 확산에 북미 투어 연기,그룹 방탄소년단BTS이 코로나19 확산으로 북미 투어 일정을 연기한다. 소속사 빅...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[그룹, 방탄, 코로나, 확산, 북미, 투어, 일정, 소속사, 빅히트엔터테인먼트, ..."
8,경향신문,2020-04-09,방탄소년단 음반 누적판매 2032만장한국 가요 최다 기록,그룹 방탄소년단사진이 앨범 누적 판매량 2000만장을 넘기면서 한국 가요 역사상 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[그룹, 방탄소년단사진, 앨범, 누적, 판매량, 한국, 역사상, 음반, 가수, 이름..."
9,조선일보,2020-05-18,이태원 아이돌 BTS 정국 아스트로 차은우 NCT 재현 세븐틴 민규였다,최근 소문으로만 떠돌던 이태원발 코로나 집단 감염 기간 이태원 방문 아이돌이 그룹...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[최근, 소문, 이태원, 코로나, 집단, 감염, 기간, 이태원, 방문, 아이돌, 그..."


### Clustering with K-means

In [99]:
# n = number of clusters
def kmeans (df, n):
    # parsing
    tfidf = TfidfVectorizer()
    article_tfidf = tfidf.fit_transform(df['tokenized_sents'].astype(str))
    article_tfidf

    # apply kmeans clustering
    n_clusters = int(n)
    km_cluster = KMeans(n_clusters=n_clusters, random_state=42)
    km_cluster.fit(article_tfidf)
    cluster_label = km_cluster.labels_
    cluster_centers = km_cluster.cluster_centers_
    
    # set a new column
    df['cluster_label'] = cluster_label
    
    # get representative articles from each clusters
    top_list = []
    for i in range(int(n)):
        d = km_cluster.transform(article_tfidf)[:,i]
        ind = np.argsort(d)[::][:1]
        top_list.append(ind[0])
    
    # re-sort by date
    df = df.iloc[top_list].sort_values(by='date')
    kmean_df = df.drop(['tokenized_sents','cluster_label'], axis = 1).reset_index(drop = True)
    return kmean_df

In [100]:
kmean_df = kmeans (tokenized_df, 10)
kmean_df

,media,date,title,article_original,url
0,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔 7MAP OF THE SOUL 7...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,경향신문,2020-06-08,BTS 유튜브 주최 가상 졸업식 디어 클래스 오브 2020 축사 꽃다발은 없어도 가...,그룹 방탄소년단BTS이 코로나19 확산으로 오프라인 졸업식에 가지 못하는 졸업생들...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,경향신문,2020-06-15,BTS 온라인 콘서트 방방콘 The Live로 107개 지역 75만 아미 만났다,그룹 방탄소년단이 실시간 라이브 공연 방방콘 The Live를 열어 75만 660...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,경향신문,2020-09-15,방탄소년단 미국 빌보드 싱글 차트 2위3주째 최상위권,그룹 방탄소년단BTS의 다이너마이트Dynamite가 미국 빌보드 메인 싱글 차트에...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-09-17,방탄소년단 빅히트 공모주 다이너마이트 터지나,내달 56일 코스피 일반 청약 신주 713만주 중 142만주 배정 청약증거금 58...,https://news.naver.com/main/read.nhn?mode=LSD&...
5,조선일보,2020-09-20,다이너마이트 영어로 부른 BTS 방탄 노래는 들려 따라갈 수 있겠다 영상,문재인 대통령이 방탄소년단BTS의 노래에 대해 그들의 노래와 춤 모두를 좋아한다고...,https://news.naver.com/main/read.nhn?mode=LSD&...
6,중앙일보,2020-10-07,아미도 냅두라는데BTS 병역특례 2년만에 다시 띄우는,BTS를 병역특례 해주자는 입장이 아니다. 순수예술인은 특례대상인데 대중예술인...,https://news.naver.com/main/read.nhn?mode=LSD&...
7,경향신문,2020-10-13,병무청 대중문화우수자 병역 연기 입법 추진확정시 BTS 연기 가능,병무청이 13일 대중문화예술 분야 우수자의 병역 징집소집 연기 등을 담은 병역법 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
8,중앙일보,2020-10-13,중국엔 정치적 지뢰 있다BTS 지우기에 깜짝 놀란 외신,방탄소년단BTS의 밴플리트상 수상 소감을 두고 중국발 논란이 커지면서 주요 외신들...,https://news.naver.com/main/read.nhn?mode=LSD&...
9,경향신문,2020-11-25,방탄소년단 또 새 역사 썼다그래미상 베스트 팝 그룹 퍼포먼스 후보,그룹 방탄소년단BTS이 미국 최고 권위 음악시상식인 그래미 어워즈 후보에 올랐다....,https://news.naver.com/main/read.nhn?mode=LSD&...


# Summarization

In [101]:
from transformers import *
from summarizer import Summarizer
from gensim.summarization.summarizer import summarize
import kss
from lexrankr import LexRank

#### Summarizer + KoBERT

In [102]:
# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained('monologg/kobert')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('monologg/kobert')
custom_model = AutoModel.from_pretrained('monologg/kobert', config=custom_config)

summarize_model = Summarizer(custom_model=custom_model, custom_tokenizer = custom_tokenizer)

In [103]:
def kobertsum (df):
    df['kobert'] = df.apply(lambda row: summarize_model(row['article_original'], ratio = 0.2), axis = 1)
    return df

In [104]:
kobert_df = kobertsum(kmean_df)

In [105]:
kobert_df

,media,date,title,article_original,url,kobert
0,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔 7MAP OF THE SOUL 7...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔 7MAP OF THE SOUL 7이...
1,경향신문,2020-06-08,BTS 유튜브 주최 가상 졸업식 디어 클래스 오브 2020 축사 꽃다발은 없어도 가...,그룹 방탄소년단BTS이 코로나19 확산으로 오프라인 졸업식에 가지 못하는 졸업생들...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단BTS이 코로나19 확산으로 오프라인 졸업식에 가지 못하는 졸업생들을...
2,경향신문,2020-06-15,BTS 온라인 콘서트 방방콘 The Live로 107개 지역 75만 아미 만났다,그룹 방탄소년단이 실시간 라이브 공연 방방콘 The Live를 열어 75만 660...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단이 실시간 라이브 공연 방방콘 The Live를 열어 75만 6600...
3,경향신문,2020-09-15,방탄소년단 미국 빌보드 싱글 차트 2위3주째 최상위권,그룹 방탄소년단BTS의 다이너마이트Dynamite가 미국 빌보드 메인 싱글 차트에...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단BTS의 다이너마이트Dynamite가 미국 빌보드 메인 싱글 차트에서...
4,조선일보,2020-09-17,방탄소년단 빅히트 공모주 다이너마이트 터지나,내달 56일 코스피 일반 청약 신주 713만주 중 142만주 배정 청약증거금 58...,https://news.naver.com/main/read.nhn?mode=LSD&...,내달 56일 코스피 일반 청약 신주 713만주 중 142만주 배정 청약증거금 58조...
5,조선일보,2020-09-20,다이너마이트 영어로 부른 BTS 방탄 노래는 들려 따라갈 수 있겠다 영상,문재인 대통령이 방탄소년단BTS의 노래에 대해 그들의 노래와 춤 모두를 좋아한다고...,https://news.naver.com/main/read.nhn?mode=LSD&...,문재인 대통령이 방탄소년단BTS의 노래에 대해 그들의 노래와 춤 모두를 좋아한다고 ...
6,중앙일보,2020-10-07,아미도 냅두라는데BTS 병역특례 2년만에 다시 띄우는,BTS를 병역특례 해주자는 입장이 아니다. 순수예술인은 특례대상인데 대중예술인...,https://news.naver.com/main/read.nhn?mode=LSD&...,순수예술인은 특례대상인데 대중예술인은 아닌 것에 대한 형평성 문제 사례로 제기한 것...
7,경향신문,2020-10-13,병무청 대중문화우수자 병역 연기 입법 추진확정시 BTS 연기 가능,병무청이 13일 대중문화예술 분야 우수자의 병역 징집소집 연기 등을 담은 병역법 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,병무청이 13일 대중문화예술 분야 우수자의 병역 징집소집 연기 등을 담은 병역법 개...
8,중앙일보,2020-10-13,중국엔 정치적 지뢰 있다BTS 지우기에 깜짝 놀란 외신,방탄소년단BTS의 밴플리트상 수상 소감을 두고 중국발 논란이 커지면서 주요 외신들...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단BTS의 밴플리트상 수상 소감을 두고 중국발 논란이 커지면서 주요 외신들도...
9,경향신문,2020-11-25,방탄소년단 또 새 역사 썼다그래미상 베스트 팝 그룹 퍼포먼스 후보,그룹 방탄소년단BTS이 미국 최고 권위 음악시상식인 그래미 어워즈 후보에 올랐다....,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단BTS이 미국 최고 권위 음악시상식인 그래미 어워즈 후보에 올랐다. ...


#### Gensim TextRank

In [106]:
# text (str) – 요약할 테스트.
# ratio (float, optional) – 요약에 대해 선택할 원본 텍스트의 문장 수 비율을 결정하는 0~1 사이 숫자.
# word_count (int or None, optional) – 출력에 포함할 단어 수. 두 파라미터가 모두 제공되는 경우 ratio는 무시된다.
# split (bool, optional) – True면 문장 list가 반환된다. False는 조인(join)된 문자열이 반환된다.
def gensimsum (df):
    df['gensim'] = df['article_original'].apply(lambda x : summarize(' '.join(kss.split_sentences(x)), word_count = 70))
    return df

In [107]:
summarized_df = gensimsum(kobert_df)

In [108]:
# 저장
summarized_df.to_csv ('./data/summarized_news.csv', index = False)

### 최종 비교 KoBERT vs Gensim

In [110]:
# for i in range(len(summarized_df)):
#      print (i,'==================================\n < ',summarized_df['title'][i],' > \n\n\n',summarized_df['date'][i],'\n', summarized_df['kobert'][i],'\n\n\n', summarized_df['gensim'][i] , '\n\n')
for i in range(len(summarized_df)):
     print (i,'==================================\n < ',summarized_df['title'][i],' > \n\n\n',summarized_df['date'][i],'\n', summarized_df['kobert'][i], '\n\n', summarized_df['gensim'][i], '\n\n\n')

0 ==================================
 <  BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹  > 


 2020-03-02 
 그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔 7MAP OF THE SOUL 7이 영국 오피셜 차트에 이어 빌보드까지 양대 팝 차트 1위에 올랐다. 빌보드가 인용한 닐슨뮤직 데이터의 자료를 보면 지난달 21일 발매된 이 앨범은 27일까지 총 42만2000장의 판매고를 올렸다. 이외에 스트리밍 횟수를 앨범 판매량으로 환산한 수치가 4만8000장 디지털 음원 다운로드 횟수를 앨범 판매량으로 환산한 수치는 2만6000장으로 집계됐다. 이후 2018년 9월 러브 유어셀프 결 앤서 지난해 4월 맵 오브 더 솔 페르소나에 이어 이번 앨범까지 1년9개월 만에 4회 연속 빌보드 200 1위를 차지했다. 

 한국 가수로서는 최초로 오피셜 차트에서 두 번 연속 정상에 오른 것은 물론 빌보드에서는 비틀스 이후 최단기간 동안 4개 앨범을 1위에 올려놓은 가수라는 기록을 쓰게 됐다.
이번 앨범은 지난달 28일 영국독일프랑스 등의 오피셜 앨범차트 1위에 올랐고 일본 오리콘차트 1위도 차지했다.
실물 음반 판매량에다 스트리밍 횟수와 디지털 음원 다운로드 횟수를 각각 앨범 판매량으로 환산한 수치를 합산해 순위를 매긴다.
빌보드는 방탄소년단이 영국의 전설적인 밴드 비틀스 이래 최단기간 동안 빌보드 200에서 4개 앨범 1위를 달성한 가수가 됐다고 전했다. 



1 ==================================
 <  BTS 유튜브 주최 가상 졸업식 디어 클래스 오브 2020 축사 꽃다발은 없어도 가장 특별한 날우리가 위로와 희망이 되고 싶다  > 


 2020-06-08 
 그룹 방탄소년단BTS이 코로나19 확산으로 오프라인 졸업식에 가지 못하는 졸업생들을 위해 축사를 했다. BTS의 리더 RM은 2020년은 참 많은 일들이 있었지만 그래도 여러분들은 해냈다며 비록 꽃다발과 학사모는 없지만 오늘은 역사상